<a href="https://colab.research.google.com/github/MarilynDaiana/Programacion_Concurrente/blob/tp/pythonTP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile archivo.py
nuevo_archivo = open("archivo_origen.txt","w")
nuevo_archivo.write("123456\n")
nuevo_archivo.write("123456hola como estan todos")

Overwriting archivo.py


In [ ]:
!chmod 700 ./archivo.py

In [ ]:
!nohup python ./archivo.py 1>salida 2>/dev/null &

In [2]:
%%writefile proceso.py
from multiprocessing import Pipe
import os
import sys

CHUNK = 512
PATH_INI = "archivo_origen.txt"
PATH_FIN = "archivo_origen"
LECTURA = "rb"
ESCRITURA = "wb"
COPIA = "copia.txt"

def padre_envia(envio,recepcion,path,modo,chunk):
  recepcion.close()

  with open(path,modo) as f:
    while True:
      datos_bytes = f.read(chunk)
      if not datos_bytes:
        break
      envio.send_bytes(datos_bytes)

  envio.close()
  os.wait()

def hijo_lee(envio,recepcion,path,modo):
  envio.close()
  new_path = path + COPIA
  datos = recepcion.recv_bytes()

  with open(path,modo) as f:
      f.write(datos)
  print("Archivo recibido")
  recepcion.close()
  os._exit(0)


#comienza el main
recepcion, envio = Pipe( False )

pid = os.fork()

if pid < 0:
  print("Error al crear nuevo proceso")
  sys.exit(1)

if pid:
  #el padre envia archivo
  padre_envia(envio,recepcion,PATH_INI,LECTURA,CHUNK)
else:
  #el hijo recibe el archivo
  hijo_lee(envio,recepcion,PATH_FIN,ESCRITURA)

Overwriting proceso.py


In [3]:
!chmod 700 ./archivo.py

In [4]:
!nohup python ./archivo.py 1>salida 2>/dev/null &

In [5]:
!diff archivo_origen.txt archivo_origencopia.txt